<a href="https://colab.research.google.com/github/swatidamele/Jupyter-Notebooks/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
from random import randrange
import random

In [ ]:
NEG = 0
POS = 1
ALL = 2

In [ ]:
df=pd.read_csv(r"../input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt",delimiter="\t",header=None,names=["IMDB Review","Sentiment"])
df.columns

FileNotFoundError: [Errno 2] No such file or directory: '../input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt'

In [ ]:
df=df.replace(',','',regex=True).replace('!','',regex=True).replace('-','',regex=True).replace('.','',regex=True)
df    

In [ ]:
split_1 = int(0.8 * len(df))
split_2 = int(0.9 * len(df))
train_data = df[:split_1]
dev_data = df[split_1:split_2]
test_data = df[split_2:]
print("Train Data")
print((train_data))
print("Dev Data")
print((dev_data))
print("Test Data")
print((test_data))

In [ ]:
# rawData = open('imdb_labelled.txt').read()

In [ ]:
# l = ['.',',','!','(',')','-','[',']','{','}',';',':','<','>','/','?','@','#','$','%','^','&','*','_','~']
# validation_data = [ x.replace(y,'')  for x in validation_data_temp for y in l if y in x ]
# print(validation_data)

                       

In [ ]:
validation_data=train_data.values.tolist()


In [ ]:
def count_label(data):
    label_count=[0,0, len(data)] # [neg_count, pos_count, all_count]
    for item in data:
        #label_count[ALL]+=1
        if item[1]== 0:
            label_count[NEG]+=1
        else:
            label_count[POS]+=1
            
    return label_count

In [ ]:
def word_vocabulary(data,threshold):
   
    vocab={}
    
    for sent in data:
        count=0
        for content in sent:
            if(count%2==0): 
                word_list = content.split()
                #print(word_list+"\n")
                for word in word_list:
                    #print(word)
                    if word.lower() not in vocab:
                        vocab[word.lower()]=[0,0,0]
                    else:
                        vocab[word.lower()][ALL]+=1
                        #print(vocab[word.lower()][POS])
                        if(sent[count+1]==1):
                            vocab[word.lower()][POS]+=1
                            #print("testPOS")
                        else:
                            if(sent[count+1]==0):
                                vocab[word.lower()][NEG]+=1
                                #print("testPOS")
                count+=1
                    
    for key in list(vocab.keys()):
        #print(key)
        if vocab[key][ALL]<threshold:
            del vocab[key]
            
    #print(vocab.values())            
    return(vocab)

In [ ]:
total_count = count_label(validation_data)
print("Count of sentences [Negative,Positive,All] is ",total_count)

In [ ]:
Dict = word_vocabulary(validation_data,5)
df_dict = pd.DataFrame(list(Dict.items()),columns = ['Word','Count [Negative,Positive,All]'])  
print(df_dict)

In [ ]:
P_word = {}

for word in Dict:
    word_count = int(Dict[word][ALL])
    total_all = total_count[ALL]
    P = word_count/total_all
    P_word[word] = P
df_p_word = pd.DataFrame(list(P_word.items()),columns = ['Word','Probability(Word)'])     
print(df_p_word) 

In [ ]:
P_conditional_pos_word = {}

for word in Dict:
    word_count = int(Dict[word][POS])
    total = total_count[POS]
    P = word_count/total
    P_conditional_pos_word[word.lower()] = P
df_p_conditional_pos_word = pd.DataFrame(list(P_conditional_pos_word.items()),columns = ['Word','Probability(Word|Positive)'])      
print(df_p_conditional_pos_word)            

In [ ]:
P_conditional_neg_word = {}

for word in Dict:
    word_count_n = int(Dict[word][NEG])
    total_n = total_count[NEG]
    P_n = word_count_n/total_n
    P_conditional_neg_word[word.lower()] = P_n
df_p_conditional_neg_word = pd.DataFrame(list(P_conditional_pos_word.items()),columns = ['Word','Probability(Word|Negative)'])     
print(df_p_conditional_neg_word)         

In [ ]:

k=5

folds=validation_data 
#print(validation_data)
fold_size= int((len(validation_data))/5)
#print(fold_size)
# for j in range(k):
#     fold=[]
#     for i in range(fold_size):
#         random_index=randrange(50)
#         if len(validation_data)>0:
#             #print(validation_data[random_index][0])
#             fold.append(validation_data[random_index][0])
#             #fold.append(random.choice(validation_data))
#             #print(fold)
#             #fold.append(newData_validate.pop(random_index))
#             #print(i)
#     folds.append(fold)
#     #print(folds)

train_fold = folds[:450]
test_fold = folds[450:]



# fold_count = 0    
# for fold in folds:
#     if(fold_count == 4):
#         test_fold = fold
#     else:
#         train_fold = train_fold + fold
#     fold_count+=1
    #train_fold = folds
    #train_fold.remove(fold)
    #train_fold = sum(train_fold, [])
    #test_fold = fold
#print(train_fold)
   





In [ ]:
def predict(review, vocabulary, label_count, lam):

    probability = []
    

    for label in[0,1]:
        prob = 1.0
        counter = 0
        for row in review:
            if(counter%2==0):
                sentarray = row.lower().split()
                for word in sentarray:
                    #print(word)
                    
                    if word not in vocabulary:
                        #print("not found")
                        continue
                    if lam == 0 and vocabulary[word][label]==0:
                        prob = 0
                        #print("zero")
                        break
                    #word_count_value = vocabulary[word][label]
                    prob = round(prob*((vocabulary[word][label]+lam) / (label_count[label]+(lam*len(vocabulary)))),5)
                     
                counter+=1
        #print(prob)        
        probability.append(prob)  
        #probability[label] = prob[label]*(label_count[label]/label_count[ALL])
    #print(probability[0],probability[1])
    #print(probability[NEG],probability[POS])
    return 0 if probability[NEG]>probability[POS] else 1
   

In [ ]:
l = [0,1]

#lam=0   

occurrence_shreshold = 5

new_train_fold = []


                    
            
#print(new_train_fold)


voca = word_vocabulary(train_fold, occurrence_shreshold)

label_count = count_label(train_fold)

for lam in l:
    accuracy = []
    correct = 0
    for review in test_fold:
        flag1 = predict(review, voca, label_count, lam)
        #print(flag1)
        flag2 = int(review[1])
        #print(flag1,flag2)
        if(flag1 == flag2):
            correct+=1
    accuracy.append(correct/len(test_fold))
    if(lam==0):
        print("Accuracy without smoothing is : ",accuracy)
    else:
        print("\nAccuracy with smoothing is : ",accuracy)

In [ ]:
def count_all_pos_Neg_data(data):
    label_count=[0,0, len(data)] # [neg_count, pos_count, all_count]
    for row in data:
        if row[1]==0:
            label_count[NEG]+=1
            #print(label_count[NEG])
        else:
            label_count[POS]+=1  
    #print(label_count)
    return label_count

In [ ]:
def Prob_by_Sentiment(data,word,senti):
    countarr=count_all_pos_Neg_data(data)
    finalarr=[]
    count=0
    count_conditional=0
    if(senti==0):
        count=countarr[1]
        snt = 0
    else:
        count=countarr[0]
        snt = 1
    
    #print(count)
    for row in data:
        if(row[1]==snt):
            sentencearr=row[0].lower().split()
            if word.lower() in sentencearr:
                count_conditional+=1
    finalarr.append(count_conditional)
    finalarr.append(countarr[0])
    finalarr.append(countarr[1])
    if(senti==0):
        cond_prob=count_conditional/countarr[0]
    else:
        cond_prob=count_conditional/countarr[1]
    #print(cond_prob)    
    return(cond_prob)

In [ ]:
def prob_word(data,word):
    count=0
    for sent in data:
        sentarr=sent[0].lower().split()
        if word.lower() in sentarr:
            count+=1
    prob=(count)/len(data)
    #print(prob)
    return prob

In [ ]:
def get_top_10(data, occurrence_shreshold):
    vocabulary = word_vocabulary(data, occurrence_shreshold)
    label_count = count_all_pos_Neg_data(data)

    word_predict = {}

    for row in data:
        for word in row[0].split():             
            #print(word)
            if word not in vocabulary: continue
            neg_prob = (Prob_by_Sentiment(data,word, NEG) * (label_count[NEG]/label_count[ALL]))/ prob_word(data,word)
            #print(neg_prob)
            pos_prob = (Prob_by_Sentiment(data,word,POS) * (label_count[POS]/label_count[ALL]))/ prob_word(data,word)
            #print(neg_prob,pos_prob)
            pred = NEG if neg_prob > pos_prob else POS
            if word not in word_predict:
                word_predict[word] = [0,0,0]
            word_predict[word][ALL] += 1
            
            if pred == int(row[1]):  
                word_predict[word][pred] += 1
            
    top_neg_lst = sorted(word_predict, key = lambda x: word_predict[x][NEG]/word_predict[x][ALL], reverse=True) [:10]
    top_pos_lst = sorted(word_predict, key = lambda x: word_predict[x][POS]/word_predict[x][ALL], reverse=True)[:10]
    finalarr=[]
    finalarr.append(top_neg_lst)
    finalarr.append(top_pos_lst)
    return finalarr

In [ ]:
arr = get_top_10(validation_data, 5)
print("Top 10 words that predict negative class are as follows: \n", arr[0])
print("\nTop 10 words that predict positive class are as follows: \n", arr[1])